In [143]:
import pandas as pd, numpy as np
import os, json
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, recall_score, confusion_matrix, accuracy_score, f1_score, precision_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
import xgboost as xgb

In [144]:
# Toggle option
SNV = False
FEATURE_SELECT = False
FEATURE_SELECTv2 = False
BALANCED = True
MY_BALANCED = True
TEST = True

# Choose dataset
DATASET = 'raw'
if TEST:
    DATASET = 'test'
elif BALANCED:
    DATASET = 'balanced'
    if SNV:
        DATASET = 'snv_balanced'
elif SNV:
    DATASET = 'snv_raw'
    if FEATURE_SELECT:
        DATASET = 'snv_feature_select'
    elif FEATURE_SELECTv2:
        DATASET = 'snv_feature_selectv2'
elif FEATURE_SELECT:
    DATASET = 'feature_select'
elif FEATURE_SELECTv2:
    DATASET = 'feature_selectv2'

DATASET

'test'

In [145]:
if BALANCED:
    # x_train
    training_data = pd.read_csv('../data/balanced_data/train_data.csv', header = None)
    # y_train
    training_labels = pd.read_csv('../data/balanced_data/train_label.csv', header = None)
    # x_test
    testing_data = pd.read_csv('../data/balanced_data/test_data.csv', header = None)
    # y_test
    testing_labels = pd.read_csv('../data/balanced_data/test_label.csv', header = None)
else:
    # x_train
    training_data = pd.read_csv('../data/original_data/noExclusion_train_data.csv', header = None)
    # y_train
    training_labels = pd.read_csv('../data/original_data/noExclusion_train_label.csv', header = None)
    # x_test
    testing_data = pd.read_csv('../data/original_data/noExclusion_test_data.csv', header = None)
    # y_test
    testing_labels = pd.read_csv('../data/original_data/noExclusion_test_label.csv', header = None)



In [146]:
print(f"training_data: {type(training_data)}, \ntraining_labels: {type(training_labels)}, \ntesting_data: {type(testing_data)}, \ntesting_labels: {type(testing_labels)}")
print(f"training labels vc: \n{training_labels.value_counts()}, \ntesting labels vc: \n{testing_labels.value_counts()}")
print(len(training_data), len(training_labels), len(testing_data), len(testing_labels))

training_data: <class 'pandas.core.frame.DataFrame'>, 
training_labels: <class 'pandas.core.frame.DataFrame'>, 
testing_data: <class 'pandas.core.frame.DataFrame'>, 
testing_labels: <class 'pandas.core.frame.DataFrame'>
training labels vc: 
2    177
3    177
1    119
dtype: int64, 
testing labels vc: 
2    59
3    59
1    40
dtype: int64
473 473 158 158


In [147]:
# type cast labels to ints
training_labels[0] = training_labels[0].astype(int)
# testing_labels
testing_labels[0] = testing_labels[0].astype(int)

# encode labels, using sklearn, to pass to xgboost
# this code was inspired by the snippet from:
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
le = LabelEncoder()
# fit the classes to the encoder and transform labels to be 0,1,2
training_labels = le.fit_transform(training_labels[0].to_list())
testing_labels = le.fit_transform(testing_labels[0].to_list())

# gridsearch results
gs_results = {}

In [148]:
# apply SNV to training data - inspired by code from my ML CW
# (Hamzah Hafejee, 2022, COMP3611_Coursework_Assessment.ipynb, Comp 3611, University of Leeds)
# (Anon, 2023b)
if SNV == True:
    print(type(training_data))
    # fit to training data
    scaler = StandardScaler().fit(training_data)
    training_data = scaler.transform(training_data)
    testing_data = scaler.transform(testing_data)
    print("After: \n", type(training_data))
    len(training_data)
else:
    print('no SNV standardisation')


no SNV standardisation


In [149]:
# RUN THIS TO APPLY FEATURE SELECTION TO TRAINING DATA
if FEATURE_SELECT == True:
    ADD_POSSIBLE_FIGURES = True

    # figures contain features from (figure_num*4)+1 
    selected_figures = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,
                        21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,
                        39,40,41,42,43,44,45,46,47,48,49,50,51,52]

    # figures that MAY be decent - noisy but different peaks
    possible_figures = [53,54,56,57,58,59,60,61,62,63,64,65]

    # decent features - eyeballed

    # generate set of selected features
    selected_features = []

    for figure_num in selected_figures:
        for i in range(0,4):
            # print(f"figure: {figure_num}, feature: feature_{(figure_num*4)+i}")
            selected_features.append((figure_num*4)+i)

    # to add possible features
    if ADD_POSSIBLE_FIGURES == True:
        for figure_num in possible_figures:
            for i in range(0,4):
                # print(f"figure: {figure_num}, feature: feature_{(figure_num*4)+i}")
                selected_features.append((figure_num*4)+i)

    # selected features round 2
    selected_features = []
    a = np.arange(30,85).tolist()
    b = np.arange(203,235).tolist()

    selected_features = np.concatenate([a,b]).tolist()
    print(selected_features)
    # (Cena, 2018)
    training_data = training_data[selected_features]
    testing_data = testing_data[selected_features]
else:
    print('no feature selection')

no feature selection


In [150]:
# make CART classifier
clf_cart = tree.DecisionTreeClassifier(criterion="gini", random_state=1)
# find optimal parameter values for CART
params = {
    'max_depth': [None, 5, 10, 15, 20, 25, 30, 35, 40], # control overfitting,
    'max_features': [None, 'sqrt', 'log2'] # performance 
}

if DATASET == 'raw':
    # raw dataset
    params = {
        'max_depth': [None], # control overfitting,
        'max_features': ['log2'] # performance 
    }
elif DATASET == 'feature_select':
    # selected + possible features
    params = {
        'max_depth': [5], # control overfitting,
        'max_features': [None] # performance 
    }
elif DATASET == 'feature_selectv2':
    # feature select v2
    params = {'max_depth': [None], 'max_features': ['sqrt']} 
elif DATASET == 'snv_raw':
    # SNV + raw
    params = {'max_depth': [None], 'max_features': ['log2']} 
elif DATASET == 'balanced':
    params = {'max_depth': [10], 'max_features': ['sqrt']} 
elif DATASET == 'snv_balanced':
    params = {'max_depth': [10], 'max_features': ['sqrt']} 

grid_search = GridSearchCV(clf_cart, params, scoring='accuracy', cv=10)
grid_search.fit(training_data, np.ravel(training_labels))
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"params = {best_params} \nbest_score: {best_score}")
best_cart = grid_search.best_estimator_
gs_results['CART'] = {'accuracy':best_score, 'params':best_params}

params = {'max_depth': None, 'max_features': 'sqrt'} 
best_score: 0.782136524822695


In [151]:
DATASET

'test'

In [152]:
# make Logistic Regressor
clf_lr = LogisticRegression(random_state=1, max_iter=1000)
params = {
    'penalty': ['l1', 'l2'], # type of regularisation 
    'C': [0.1, 1, 10, 100], # regularisation strength
    'solver': ['liblinear', 'saga', 'lbfgs', 'newton-cg'] # approach to finding best weights
}

if DATASET == 'raw':
    # raw dataset
    params = {
        'penalty': ['l2'], # type of regularisation 
        'C': [0.1], # regularisation strength
        'solver': ['lbfgs'] # approach to finding best weights
    }
elif DATASET == 'feature_select':
    # selected + possible features
    params = {
        'penalty': ['l2'], # type of regularisation 
        'C': [0.1], # regularisation strength
        'solver': ['newton-cg'] # approach to finding best weights
    }
elif DATASET == 'feature_selectv2':
    # FSv2
    params = {'C': [1], 'penalty': ['l2'], 'solver': ['lbfgs']} 
elif DATASET == 'snv_raw':
    # SNV + raw
    params = {'C': [0.1], 'penalty': ['l2'], 'solver': ['lbfgs']} 
elif DATASET == 'balanced':
    params = {'C': [100], 'penalty': ['l2'], 'solver': ['saga']} 
elif DATASET == 'snv_balanced':
    params = {'C': [0.1], 'penalty': ['l2'], 'solver': ['saga']} 

grid_search = GridSearchCV(clf_lr, params, scoring='accuracy', cv=10)
grid_search.fit(training_data, np.ravel(training_labels))
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"params = {best_params} \nbest_score: {best_score}")
gs_results['LR'] = {'accuracy':best_score, 'params':best_params}


c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\sc19mhh\Desktop

params = {'C': 10, 'penalty': 'l2', 'solver': 'saga'} 
best_score: 0.7460106382978724


c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [153]:
print(f"params = {best_params} \nbest_score: {best_score}")
best_lr = grid_search.best_estimator_

params = {'C': 10, 'penalty': 'l2', 'solver': 'saga'} 
best_score: 0.7460106382978724


In [154]:
# make Gaussian Naive Bayes classifier
clf_nb = GaussianNB()
params = {
    'var_smoothing':[1e-20, 1e-19, 1e-18, 1e-17, 1e-16, 1e-15, 1e-14, 1e-13, 1e-12, 1e-11, 1e-10, 1e-9], # from less smoothing to more aggressive smoothing
}
grid_search = GridSearchCV(clf_nb, params, scoring='accuracy', cv=10)
grid_search.fit(training_data, np.ravel(training_labels))
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"params = {best_params} \nbest_score: {best_score}")
best_nb = grid_search.best_estimator_
gs_results['GNB'] = {'accuracy':best_score, 'params':best_params}

params = {'var_smoothing': 1e-20} 
best_score: 0.663918439716312


In [155]:
# make k-Nearest Neighbours classifier
clf_knn = KNeighborsClassifier(n_jobs=-1) # use all processes for parellelisation
params = {
    'n_neighbors': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
}
grid_search = GridSearchCV(clf_knn, params, scoring='accuracy', cv=10)
grid_search.fit(training_data, np.ravel(training_labels))
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"params = {best_params} \nbest_score: {best_score}")
best_knn = grid_search.best_estimator_
gs_results['kNN'] = {'accuracy':best_score, 'params':best_params}

params = {'n_neighbors': 1} 
best_score: 0.8391400709219858


In [156]:
# make SVM-RBF classifier
clf_svmrbf = SVC(kernel='rbf', random_state=1)
params = {
    'C': [0.1, 1, 10, 100, 1000], # high to low regularisation strength
    'gamma' : ['scale', 'auto'], # need to research this parameter more
}

if DATASET == 'raw':
    # raw dataset
    params = {
        'C': [100], # high to low regularisation strength
        'gamma' : ['scale'], # need to research this parameter more
    }

elif DATASET == 'feature_select':
    # selected + possible features
    params = {
        'C': [1000], # high to low regularisation strength
        'gamma' : ['scale'], # need to research this parameter more
    }

elif DATASET == 'feature_selectv2':
    # selected + possible features v2
    params = {
        'C': [1000], # high to low regularisation strength
        'gamma' : ['scale'], # need to research this parameter more
    }

elif DATASET == 'snv_raw':
    # SNV + raw
    params = {'C': [10], 'gamma': ['scale']} 

elif DATASET == 'balanced':
    params = {'C': [1000], 'gamma': ['scale']} 
    
elif DATASET == 'snv_balanced':
    params = {'C': [1000], 'gamma': ['scale']}

grid_search = GridSearchCV(clf_svmrbf, params, scoring='accuracy', cv=10)
grid_search.fit(training_data, np.ravel(training_labels))
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"params = {best_params} \nbest_score: {best_score}")
best_svmrbf = grid_search.best_estimator_
gs_results['SVM-RBF'] = {'accuracy':best_score, 'params':best_params}

params = {'C': 100, 'gamma': 'scale'} 
best_score: 0.7989804964539007


In [157]:
# make SVM linear classifier
clf_lin = SVC(kernel='linear', random_state=1)
params = {
    'C': [0.1, 1, 10, 100, 1000], # high to low regularisation strength
    'gamma' : ['scale', 'auto'], # need to research this parameter more
    # 'gamma' : [], # need to research this parameter more
}

if DATASET == 'raw':
    # raw dataset
    params = {
        'C': [10], # high to low regularisation strength
        'gamma' : ['scale'], # need to research this parameter more
    }

elif DATASET == 'feature_select':
    # selected + possible features
    params = {
        'C': [1], # high to low regularisation strength
        'gamma' : ['scale'], # need to research this parameter more
    }

elif DATASET == 'feature_selectv2':
    # FSv2
    params = {
        'C': [1], # high to low regularisation strength
        'gamma' : ['scale'], # need to research this parameter more
    }

elif DATASET == 'snv_raw':
    # SNV + raw
    params = {'C': [0.1], 'gamma': ['scale']} 

elif DATASET == 'balanced':
    params = {'C': [1], 'gamma': ['scale']} 

elif DATASET == 'snv_balanced':
    params = {'C': [0.1], 'gamma': ['scale']} 

grid_search = GridSearchCV(clf_lin, params, scoring='accuracy', cv=10)
grid_search.fit(training_data, np.ravel(training_labels))
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"params = {best_params} \nbest_score: {best_score}")
best_svmlin = grid_search.best_estimator_
gs_results['SVM-Lin'] = {'accuracy':best_score, 'params':best_params}

params = {'C': 10, 'gamma': 'scale'} 
best_score: 0.7460106382978723


In [158]:
# make svm sigmoidal classifier
clf_sig = SVC(kernel='sigmoid', random_state=1)
params = {
    'C': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10], # high to low regularisation strength
    'gamma' : ['scale', 'auto'], # need to research this parameter more
    # 'gamma' : [], # need to research this parameter more
}

if DATASET == 'raw':
    # raw dataset
    params = {
        'C': [10], # high to low regularisation strength
        'gamma' : ['auto'], # need to research this parameter more
    }

elif DATASET == 'feature_select':
    # selected + possible features
    params = {
        'C': [1e-05], # high to low regularisation strength
        'gamma' : ['scale'], # need to research this parameter more
    }

elif DATASET == 'feature_selectv2':
    # FSv2
    params = {
        'C': [0.1], # high to low regularisation strength
        'gamma' : ['auto'], # need to research this parameter more
    }

elif DATASET == 'snv_raw':
    # SNV + raw
    params = {'C': [0.1], 'gamma': ['scale']} 

elif DATASET == 'balanced':
    params = {'C': [0.1], 'gamma': ['auto']}

elif DATASET == 'snv_balanced':
    params = {'C': [0.1], 'gamma': ['scale']} 
    
grid_search = GridSearchCV(clf_sig, params, scoring='accuracy', cv=10)
grid_search.fit(training_data, np.ravel(training_labels))
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"params = {best_params} \nbest_score: {best_score}")
best_svmsig = grid_search.best_estimator_
gs_results['SVM-Sig'] = {'accuracy':best_score, 'params':best_params}

params = {'C': 10, 'gamma': 'auto'} 
best_score: 0.6277039007092198


In [159]:
# make xgboost classifier
clf_xgb = xgb.XGBClassifier(random_state = 1)

params = {
    'n_estimators': [10,100, 500, 1000], # no. boosting rounds
    'max_depth': [3,5,7,10,15] # control overfitting
}

if DATASET == 'raw':
    # raw dataset
    params = {
        'n_estimators': [100], # no. boosting rounds
        'max_depth': [10] # control overfitting
    }

elif DATASET == 'feature_select':
    # selected + possible features
    params = {
        'n_estimators': [10], # no. boosting rounds
        'max_depth': [5] # control overfitting
    }

elif DATASET == 'feature_selectv2':
    # FSv2
    params = {
        'n_estimators': [100], # no. boosting rounds
        'max_depth': [5] # control overfitting
    }

elif DATASET == 'snv_raw':
    # SNV + raw
    params = {'max_depth': [10], 'n_estimators': [100]} 

elif DATASET == 'balanced':
    params = {'max_depth': [15], 'n_estimators': [100]} 

elif DATASET == 'snv_balanced':
    params = {'max_depth': [15], 'n_estimators': [100]} 
    
grid_search = GridSearchCV(clf_xgb, params, scoring='accuracy', cv=10)
grid_search.fit(training_data, training_labels)
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"params = {best_params} \nbest_score: {best_score}")
best_xgb = grid_search.best_estimator_
gs_results['XGB'] = {'accuracy':best_score, 'params':best_params}

params = {'max_depth': 15, 'n_estimators': 100} 
best_score: 0.8266843971631206


In [160]:
# make adaboost classifier
clf_ada = AdaBoostClassifier(random_state=1)
params = {
    'n_estimators': [10, 50, 100, 500, 1000],
    'learning_rate': [0, 0.01, 0.1, 1, 10] # weight applied to each clf at each boosting iteration
}

if DATASET == 'raw':
    # raw dataset
    params = {
        'n_estimators': [50],
        'learning_rate': [0.01] # weight applied to each clf at each boosting iteration
    }

elif DATASET == 'feature_select':
    # selected + possible features
    params = {
        'n_estimators': [10],
        'learning_rate': [0.01] # weight applied to each clf at each boosting iteration
    }

elif DATASET == 'feature_selectv2':
    # FSv2
    params = {
        'n_estimators': [50],
        'learning_rate': [0.01] # weight applied to each clf at each boosting iteration
    }

elif DATASET == 'snv_raw':
    # SNV + raw
    params = {'learning_rate': [0.01], 'n_estimators': [50]} 

elif DATASET == 'balanced':
    params = {'learning_rate': [0.01], 'n_estimators': [10]} 

elif DATASET == 'snv_balanced':
    params = {'learning_rate': [0.01], 'n_estimators': [10]} 
    
grid_search = GridSearchCV(clf_ada, params, scoring='accuracy', cv=10)
grid_search.fit(training_data, training_labels)
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"params = {best_params} \nbest_score: {best_score}")
best_ada = grid_search.best_estimator_
gs_results['ADA'] = {'accuracy':best_score, 'params':best_params}

c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
50 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 124, in fit
    

params = {'learning_rate': 0.01, 'n_estimators': 100} 
best_score: 0.7062056737588651


In [161]:
# make Random Forest classifier
clf_rf = RandomForestClassifier(random_state=1)
params = {
  'n_estimators': [10, 50, 100, 200, 300],
  'max_depth': [None, 5, 10, 20, 30, 40],
  "max_features" : [None, 1, 5, 10, 20, 30]
}

if DATASET == 'raw':
  # raw dataset
  params = {
    'n_estimators': [50],
    'max_depth': [10],
    "max_features" : [30]
  }

elif DATASET == 'feature_select':
  # selected + possible features
  params = {
    'n_estimators': [200],
    'max_depth': [None],
    "max_features" : [5]
  }

elif DATASET == 'feature_selectv2':
  # feature select v2
  params = {'max_depth': [None], 'max_features': [5], 'n_estimators': [300]}

elif DATASET == 'snv_raw':
  # SNV + raw
  params = {'max_depth': [10], 'max_features': [30], 'n_estimators': [50]} 

elif DATASET == 'balanced':
  params = {'max_depth': [10], 'max_features': [5], 'n_estimators': [100]} 

elif DATASET == 'snv_balanced':
  params = {'max_depth': [10], 'max_features': [5], 'n_estimators': [100]} 
  
grid_search  = GridSearchCV(clf_rf, params, scoring='accuracy', cv=10)
grid_search.fit(training_data, np.ravel(training_labels))
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"params = {best_params} \nbest_score: {best_score}")
best_rf = grid_search.best_estimator_
gs_results['RF'] = {'accuracy':best_score, 'params':best_params}

params = {'max_depth': 10, 'max_features': 5, 'n_estimators': 300} 
best_score: 0.8329787234042554


In [162]:
# ensemble model (Anon, 2014), (Anon, 2023b)
ensemble = VotingClassifier(estimators=[
    ('rf', best_rf), 
    ('knn', best_knn), 
    ('xgb', best_xgb), 
    ('svmrbf', best_svmrbf), 
    ('nb', best_nb)],
    voting='hard',
    n_jobs=-1)

ensemble.fit(training_data, training_labels)

accuracy = ensemble.score(testing_data, testing_labels)
predictions = ensemble.transform(testing_data)
gs_results['Ensemble'] = {'accuracy':accuracy}
accuracy

0.8291139240506329

In [163]:
# sorted GS models
print(gs_results)
gs_sorted_models = dict(sorted(gs_results.items(), key=lambda item: item[1]['accuracy'], reverse=True))
print(gs_sorted_models.keys())

{'CART': {'accuracy': 0.782136524822695, 'params': {'max_depth': None, 'max_features': 'sqrt'}}, 'LR': {'accuracy': 0.7460106382978724, 'params': {'C': 10, 'penalty': 'l2', 'solver': 'saga'}}, 'GNB': {'accuracy': 0.663918439716312, 'params': {'var_smoothing': 1e-20}}, 'kNN': {'accuracy': 0.8391400709219858, 'params': {'n_neighbors': 1}}, 'SVM-RBF': {'accuracy': 0.7989804964539007, 'params': {'C': 100, 'gamma': 'scale'}}, 'SVM-Lin': {'accuracy': 0.7460106382978723, 'params': {'C': 10, 'gamma': 'scale'}}, 'SVM-Sig': {'accuracy': 0.6277039007092198, 'params': {'C': 10, 'gamma': 'auto'}}, 'XGB': {'accuracy': 0.8266843971631206, 'params': {'max_depth': 15, 'n_estimators': 100}}, 'ADA': {'accuracy': 0.7062056737588651, 'params': {'learning_rate': 0.01, 'n_estimators': 100}}, 'RF': {'accuracy': 0.8329787234042554, 'params': {'max_depth': 10, 'max_features': 5, 'n_estimators': 300}}, 'Ensemble': {'accuracy': 0.8291139240506329}}
dict_keys(['kNN', 'RF', 'Ensemble', 'XGB', 'SVM-RBF', 'CART', 'LR

In [164]:
# evaluate models (Anon, 2023b)
# model_metrics = {'accuracy', 'recall', 'precision', 'F1-score', 'ROC-AUC'}
model_metrics = {}

# all of the models
models = [best_cart, best_rf, best_lr, best_nb, best_knn, best_svmrbf, best_svmlin, best_svmsig, best_xgb, best_ada, ensemble]
model_names = ['CART', 'RF', 'LR', 'GNB', 'kNN', 'SVM-RBF', 'SVM-Lin', 'SVM-Sig', 'XGB', 'ADA', 'Ensemble']
i=0
for model in models:
    # train on test set
    predicted = model.predict(testing_data)
    # generate cm against test labels
    cm = confusion_matrix(testing_labels, predicted)
    # print(cm)
    accuracy = accuracy_score(testing_labels, predicted)
    recall = recall_score(testing_labels, predicted, average=None)
    precision = precision_score(testing_labels, predicted, average=None)
    f1 = f1_score(testing_labels, predicted, average=None)

    try:
        predicted_prob = model.predict_proba(testing_data)
        roc = roc_auc_score(testing_labels, predicted_prob, average=None, multi_class='ovr') 
        # print(accuracy, recall, precision, f1, roc)
        model_metrics[model_names[i]] = {
                                            'accuracy':accuracy, 
                                            'recall':{
                                                1:recall[0], 
                                                2:recall[1], 
                                                3:recall[2]
                                            },
                                            'precision':{
                                                1:precision[0], 
                                                2:precision[1], 
                                                3:precision[2]
                                            },
                                            'f1_score':{
                                                1:f1[0], 
                                                2:f1[1], 
                                                3:f1[2]
                                            },
                                            'ROC-AUC':{
                                                1:roc[0], 
                                                2:roc[1], 
                                                3:roc[2]
                                            }
        }
    except:
        print(f"can't predict class probilities for {model_names[i]}")
        # print(accuracy, recall, precision, f1)
        model_metrics[model_names[i]] = {
                                            'accuracy':accuracy, 
                                            'recall':{
                                                1:recall[0], 
                                                2:recall[1], 
                                                3:recall[2]
                                            },
                                            'precision':{
                                                1:precision[0], 
                                                2:precision[1], 
                                                3:precision[2]
                                            },
                                            'f1_score':{
                                                1:f1[0], 
                                                2:f1[1], 
                                                3:f1[2]
                                            }
        }
    i+=1

# (Gern Blanston, 2009)- sort by neoplasia recall
sorted_metrics = dict(sorted(model_metrics.items(), key=lambda item: item[1]['recall'][3], reverse=True))
# (holys, 2013)
with open('metrics/my_balanced_dataset/model_metrics_recall_ensemble.json', 'w') as fp:
    json.dump(sorted_metrics, fp)

# redo but sort by accuracy
# (Gern Blanston, 2009)
sorted_metrics_acc = dict(sorted(model_metrics.items(), key=lambda item: item[1]['accuracy'], reverse=True))
# (holys, 2013)
with open('metrics/my_balanced_dataset/model_metrics_accuracy_ensemble.json', 'w') as fp:
    json.dump(sorted_metrics_acc, fp)
model_metrics
sorted_metrics
 

can't predict class probilities for SVM-RBF
can't predict class probilities for SVM-Lin
can't predict class probilities for SVM-Sig
can't predict class probilities for Ensemble


{'XGB': {'accuracy': 0.8544303797468354,
  'recall': {1: 0.975, 2: 0.8305084745762712, 3: 0.7966101694915254},
  'precision': {1: 0.8666666666666667,
   2: 0.8448275862068966,
   3: 0.8545454545454545},
  'f1_score': {1: 0.9176470588235294,
   2: 0.8376068376068375,
   3: 0.8245614035087718},
  'ROC-AUC': {1: 0.9932203389830508,
   2: 0.9589111453518233,
   3: 0.9289505221708613}},
 'RF': {'accuracy': 0.8607594936708861,
  'recall': {1: 0.975, 2: 0.864406779661017, 3: 0.7796610169491526},
  'precision': {1: 0.8666666666666667,
   2: 0.8360655737704918,
   3: 0.8846153846153846},
  'f1_score': {1: 0.9176470588235294, 2: 0.85, 3: 0.8288288288288288},
  'ROC-AUC': {1: 0.9963983050847458,
   2: 0.9678137305255949,
   3: 0.9571991097414827}},
 'kNN': {'accuracy': 0.8227848101265823,
  'recall': {1: 0.925, 2: 0.7966101694915254, 3: 0.7796610169491526},
  'precision': {1: 0.925, 2: 0.7833333333333333, 3: 0.7931034482758621},
  'f1_score': {1: 0.925, 2: 0.7899159663865546, 3: 0.786324786324786

In [165]:
# print highest acc models from gridsearch
print(f"gs_sorted_models (acc): \n{gs_sorted_models.keys()}\n")

# highest acc models from test set
print(f"sorted models (acc): \n{sorted_metrics_acc.keys()}\n")

# highest recall from test set
print(f"sorted models (recall): \n{sorted_metrics.keys()}")


gs_sorted_models (acc): 
dict_keys(['kNN', 'RF', 'Ensemble', 'XGB', 'SVM-RBF', 'CART', 'LR', 'SVM-Lin', 'ADA', 'GNB', 'SVM-Sig'])

sorted models (acc): 
dict_keys(['SVM-RBF', 'RF', 'XGB', 'Ensemble', 'kNN', 'CART', 'ADA', 'LR', 'SVM-Lin', 'SVM-Sig', 'GNB'])

sorted models (recall): 
dict_keys(['XGB', 'RF', 'kNN', 'CART', 'Ensemble', 'SVM-RBF', 'LR', 'GNB', 'SVM-Lin', 'ADA', 'SVM-Sig'])


In [166]:
# (Neekhara, 2019)
import json

DATASET='my_balanced'
# function to add to JSON
def write_json(new_data, filename='metrics/scoreboard.json'):
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        # Join new_data with file_data inside emp_details
        file_data[DATASET] = (new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file, indent = 4)

# calculate avg accuracy and recall
accuracy = 0
recall = 0
count = 0
for model in model_names:
    # print(model_metrics[model]['accuracy'])
    accuracy += model_metrics[model]['accuracy']
    recall += model_metrics[model]['recall'][3]
    count +=1

t6acc = 0
t6rec = 0
count2 = 0
for key in sorted_metrics_acc:
    if count2 ==6:
        break
    accuracy += sorted_metrics_acc[key]['accuracy']
    recall += sorted_metrics_acc[key]['recall'][3]
    count2 +=1

avg = {'all' : {'accuracy': accuracy/count, 'neoplasia recall': recall/count},
       'top6' : {'accuracy': t6acc/count2, 'recall':t6rec/count2}}
print(accuracy, recall, count)
print(t6acc, t6rec, count2)
if DATASET != 'test':
    write_json(avg)
else:
    print(DATASET)
avg



13.79746835443038 12.28813559322034 11
0 0 6


{'all': {'accuracy': 1.2543153049482163,
  'neoplasia recall': 1.1171032357473036},
 'top6': {'accuracy': 0.0, 'recall': 0.0}}

# observations on raw dataset
XGBoost and ADAboost seem to have really overfit, because they severely underperform on unseen test data, compared to the accuracies they were achieving with gridsearch. IGNORE THIS: it is just because the test labels were not normalised!

Although GNB has higher recall for neoplasia than kNN, kNN seems to be the best classifier overall. While GNB has highest recall for neoplasia, has 3rd lowest accuracy.

Top models based on accuracy, from gridsearch, were RF, kNN, XGB, SVM-RBF, CART. Top models based on accuracy, from test set, were RF, kNN, SVM-RBF, CART, SVM-Lin. Therefore, RF, kNN, SVM-RBF, CART seem to perform well, in terms of accuracy, and don't seem to produce drastically different results with the test set, suggesting there isn't much overfitting

gs_sorted_models (acc): 
(['RF', 'kNN', 'XGB', 'SVM-RBF', 'CART', 'LR', 'SVM-Lin', 'ADA', 'SVM-Sig', 'GNB'])

sorted models (acc): 
(['RF', 'XGB', 'kNN', 'SVM-RBF', 'CART', 'SVM-Lin', 'LR', 'SVM-Sig', 'GNB', 'ADA'])

sorted models (recall): 
(['GNB', 'kNN', 'CART', 'RF', 'SVM-Lin', 'XGB', 'LR', 'SVM-RBF', 'SVM-Sig', 'ADA'])

# observations on feature selected dataset
Some models decreased in performance, some increased, with largest increase being 6% increase in accuracy for SVM-Lin model. But overall, not worth, since the max accuracy of any of the models was lower than without feature selection. Maybe better feature selection is needed - an analytical solution rather than eyeball

gs_sorted_models (acc): 
(['RF', 'SVM-RBF', 'kNN', 'XGB', 'CART', 'SVM-Lin', 'LR', 'ADA', 'GNB', 'SVM-Sig'])

sorted models (acc): 
(['RF', 'XGB', 'kNN', 'SVM-RBF', 'CART', 'LR', 'SVM-Lin', 'GNB', 'ADA', 'SVM-Sig'])

sorted models (recall): 
(['GNB', 'SVM-RBF', 'XGB', 'RF', 'kNN', 'SVM-Lin', 'CART', 'LR', 'SVM-Sig', 'ADA'])